In [218]:
train_dataset[idx]['images'].shape

torch.Size([3, 500, 500])

In [279]:
encoder = EfficientNet.from_pretrained('efficientnet-b0')

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth


  0%|          | 0.00/20.4M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b0


In [277]:
encoder = getattr(models, 'resnet34')(pretrained=True)
encoder

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [262]:
img = th.from_numpy(img).transpose(2, 0).to(th.double)
img

tensor([[[0.4980, 0.4980, 0.4745,  ..., 0.3686, 0.3765, 0.3804],
         [0.5020, 0.4980, 0.4784,  ..., 0.3804, 0.3843, 0.3882],
         [0.5098, 0.5059, 0.4902,  ..., 0.3961, 0.4039, 0.4078],
         ...,
         [0.4275, 0.4314, 0.4314,  ..., 0.6275, 0.6235, 0.6275],
         [0.4314, 0.4314, 0.4353,  ..., 0.6392, 0.6353, 0.6392],
         [0.4314, 0.4314, 0.4314,  ..., 0.6431, 0.6353, 0.6392]],

        [[0.5216, 0.5176, 0.4980,  ..., 0.4235, 0.4353, 0.4392],
         [0.5255, 0.5216, 0.5020,  ..., 0.4314, 0.4431, 0.4510],
         [0.5294, 0.5255, 0.5098,  ..., 0.4510, 0.4627, 0.4667],
         ...,
         [0.4471, 0.4510, 0.4627,  ..., 0.6588, 0.6549, 0.6588],
         [0.4510, 0.4510, 0.4667,  ..., 0.6627, 0.6627, 0.6627],
         [0.4471, 0.4510, 0.4627,  ..., 0.6667, 0.6627, 0.6627]],

        [[0.5294, 0.5255, 0.5098,  ..., 0.3922, 0.4000, 0.4039],
         [0.5373, 0.5294, 0.5137,  ..., 0.4000, 0.4078, 0.4078],
         [0.5451, 0.5412, 0.5255,  ..., 0.4118, 0.4196, 0.

In [280]:
encoder(img.float().unsqueeze(0)).shape

torch.Size([1, 1000])

In [ ]:
class CassavaDiseaseClassifier(pl.LightningModule):
    def __init__(self, lr, base_model, freeze, num_classes, pretrained=True):
        self.lr = lr
        self.base_model = base_model
        self.freeze = freeze
        self.num_classes = num_classes
        self.transforms = nn.sequential(
        
        )
        self.encoder = getattr(models, self.base_model)(pretrained=pretrained)
        
        if self.freeze:
            for param in self.encoder.parameters():
                param.requires_grad = False
                

        if "resnet" in self.base_model:
            try:
                self.num_ftrs = self.encoder.fc.out_features
            except:
                self.num_ftrs = self.encoder._fc.out_features 
    
        elif "efficientnet" in self.base_model:
            try:
                self.num_ftrs = self.encoder.fc.out_features 
            except:
                self.num_ftrs = self.encoder._fc.out_features 
                
        elif "densenet" in self.base_model:
            try:
                self.num_ftrs = self.encoder.classifier.out_features 
            except:
                pass
            
        self.dropout = nn.Dropout(p=.35)
        self.decoder = nn.Linear(in_features=self.num_ftrs, out_features=self.num_classes)
                
                
    def forward(self, images, targets=None):
        # extract features
        out = self.encoder(images)
        # apply dropout
        out = self.dropout(out)
        # apply classifier
        out = self.decoder(out)
        
        return out
    
    def configure_optimizers(self):
        opt = optim.Adam(params = self.parameters(), lr=self.lr)
        
        return opt
    
    def training_step(self, batch, batch_idx):
        pass
    
    
    def validation_step(self, batch, batch_idx):
        pass
    
    def predict(self, dataset, batch_size):
        pass

# Training part

# Inference/Test Part

In [33]:
ss.head()

,image_id,label
0,2216849948.jpg,4
